initialize sigma randomly with "initial" function and plot the grid using "graphic" function

In [1]:
import numpy as np
import matplotlib.pyplot as plt
def initial(l):
    p=0.5
    sigma=np.random.uniform(size=(l,l))
    z=(sigma<p).astype(int)
    z[z==0]=-1
    return z
def graphic(z,l):
    x=np.arange(l)
    y=np.arange(l)
    plt.pcolormesh(x,y,z,cmap='gray_r')

"energy" function to calculate energy of the grid

In [2]:
def energy(sigma,l):
    E=np.sum(sigma[:,:l-1]*sigma[:,1:l])+np.sum(sigma[:l-1,:]*sigma[1:l,:])+np.sum(sigma[:,l-1]*sigma[:,0])+np.sum(sigma[l-1,:]*sigma[0,:])
    return -E   

"magnetization" function to calculate magnetization of the grid

In [3]:
def magnetization(sigma):
    return np.sum(sigma)

"cor" function to calculate correlation of the "data"

In [5]:
def cor(data,l,step):
    n=len(data)
    x=0
    for i in range(step,n//3+1,step):
         x=stats.pearsonr(data[:n-i],data[i:])[0]
         if x<(1/np.e):
            return i
    return 0

"cor2" function is as same as cor function but it caculates correlation without using the standard libraries:

In [6]:
def cor2(data,l,step):
    n=len(data)
    u=[]
    data2=np.array(data)
    for i in range(step,n//10,step):
        y=(np.mean(data2[:n-i]*data2[i:])-np.mean(data2[:n-i])*np.mean(data2[i:]))/(np.std(data2[:n-i])*np.std(data2[i:]))
        u.append(y)
    return u

"sv_excel" function to save "data" which is a pandas dataframe in to an excel file 

In [7]:
from openpyxl import load_workbook
def sv_excel(data,l,fname):
    path=r"%s"%fname
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book
    data.to_excel(writer,sheet_name=str(l))
    writer.save()
    writer.close()
    

"relax" function to make sure that the system has reached to its stable state 

In [9]:
def relax(sigma,e,m,j,l,sample_number):
  des={0:1,4:(np.e)**(-j*4),8:(np.e)**(-j*8)}
  for k in range(sample_number):
    a,b=np.random.randint(0,l,size=2)
    de=2*sigma[a,b]*(sigma[(a+1)%l,b]+sigma[a-1,b]+sigma[a,(b+1)%l]+sigma[a,b-1])
    if de<0:
      sigma[a,b]*=-1
      e+=de
      m+=2*sigma[a,b]
    elif de>0 and np.random.rand()<des[de]:
      sigma[a,b]*=-1
      e+=de
      m+=2*sigma[a,b]
  return sigma,e,m

'get_corr' function to calculate the energy corrolation length 

In [ ]:
def get_corr(sigma,e,m,j,l,relax_limit):
  des={0:1,4:(np.e)**(-j*4),8:(np.e)**(-j*8)}
  cor_l=0
  count=0
  ee=[]
  while cor_l==0:
    count+=1
    #ee=[]
    for i in range(relax_limit):
      a,b=np.random.randint(0,l,size=2)
      de=2*sigma[a,b]*(sigma[(a+1)%l,b]+sigma[a-1,b]+sigma[a,(b+1)%l]+sigma[a,b-1])
      if de<0:
        sigma[a,b]*=-1
        e+=de
        m+=2*sigma[a,b]
      elif de>0 and np.random.rand()<des[de]:
        sigma[a,b]*=-1
        e+=de
        m+=2*sigma[a,b]
      ee.append(e)
    cor_l=cor(ee,l,l*l)*count
  return sigma,e,m,cor_l

'main' function to produce data:

In [10]:
def main(sigma,e,m,j,l,corrolation_length,sample_number):
  des={0:1,4:(np.e)**(-j*4),8:(np.e)**(-j*8)}
  ee=[]
  mm=[]
  spin=[]
  final_data=[]
  for k in range(sample_number):
    for i in range(corrolation_length):
        a,b=np.random.randint(0,l,size=2)
        de=2*sigma[a,b]*(sigma[(a+1)%l,b]+sigma[a-1,b]+sigma[a,(b+1)%l]+sigma[a,b-1])
        if de<0:
          sigma[a,b]*=-1
          e+=de
          m+=2*sigma[a,b]
        elif de>0 and np.random.rand()<des[de]:
          sigma[a,b]*=-1
          e+=de
          m+=2*sigma[a,b]
    ee.append(e)
    mm.append(m)
    spin.append(spin_spin(sigma))
  final_data=(np.mean(ee),np.var(ee),np.mean(abs(array(mm))/(l*l)),np.var(abs(array(mm))/(l*l)),np.mean(spin),np.std(spin))
  return sigma,final_data,e,m

plot data:

In [11]:
from openpyxl import load_workbook

def plot(file_address,specific_row,plot_title,x_label,y_label,pic_name):
    plt.style.use('seaborn')
    fname=file_address
    xls = xlrd.open_workbook(r'%s'%fname, on_demand=True)
    sheet_names=xls.sheet_names()
    sheet_names.remove('Sheet')

    fig1=plt.figure()
    ax1=fig1.add_subplot()
    style=['--o','--v','--s','--p','--P','--*']
    for counter,sheet in enumerate(sheet_names):
        df=pd.read_excel(fname,sheet_name=sheet,index_col=0)
        J=list(df.columns)
        ax1.plot(J,df.loc[specific_row],style[counter%len(style)],label='L=%s'%sheet)

    ax1.set_xlabel(x_label)
    ax1.set_ylabel(y_label)
    ax1.set_title(plot_title)
    ax1.legend()
    plt.tight_layout()
    plt.savefig('C:\\Users\\alise\Dropbox\\simu in physics\\HW-7\\plots\\%s.png'%pic_name, bbox_inches='tight') 
    plt.show()

main body:

In [ ]:
from numpy import array
import pandas as pd 
from openpyxl import Workbook


#%cd /content/drive/My\ Drive/Colab\ Notebooks/ising-data2

fname='data.xlsx'
#J=np.append(np.arange(0.1,0.4,0.05),np.arange(0.41,0.9,0.01))
J=np.append(np.arange(0.1,0.4,0.05),np.arange(0.41,0.50,0.01))
J=np.append(J,np.arange(0.55,0.75,0.05))
df=pd.DataFrame(columns=J,index= ['e','var_e','m','var_m','s-s corr','s-s std'])
L=[32]
#L=[64, 100,130, 160]
for l in L:
  print('L=%i \n ******************************************'%l)
  #J=[0.7]
  sigma=initial(l)
  e=energy(sigma,l)
  m=magnetization(sigma)
  sample_number=100
  sample_relax_number=100*l*l
  for counter,j in enumerate(J,1):
    if j>=0.40:
      sample_relax_number=1000*l*l

    sigma,e,m=relax2(sigma,e,m,j,l,sample_relax_number)
    sigma,e,m,cor_l=relax(sigma,e,m,j,l,6*(l*l))
    print('j=%0.2f \n cor=%i'%(j,cor_l))
    if j>=0.40 and j<=0.5:
      sample_number=300
    else:
      sample_number=100
    sigma,final_data,e,m=main(sigma,e,m,j,l,2*cor_l,sample_number)
      #graphic(sigma,l)
      #plt.title('J=%0.2f'%j)
      #plt.savefig('/content/drive/My Drive/Colab Notebooks/ising-pics/200/%0.2f.png'%j,bbox_inches='tight')
      #plt.pause(0.001)
      #plt.plot(np.arange(len(ee)),ee)
      #plt.pause(0.001)
    df[j]=final_data
  sv_excel(df,l,fname)

plot data: change the second parameter of the plot function to get your desired result. Causion! for the possible values check the data excel file

In [ ]:
fname='C:\\Users\\alise\\Google Drive\\Colab Notebooks\\ising-data2\\data.xlsx'
plot(fname,'s-s corr',"spin-spin correlation length vs $\\beta$",'$\\beta$', '$\\xi$',"correlation length vs beta")